# Table of Contents
* [Setup](#Setup)
	* [estimate cost](#estimate-cost)
	* [load dataset](#load-dataset)
* [Submitting HITs](#Submitting-HITs)
* [Retrieve results](#Retrieve-results)
* [Interact with workers](#Interact-with-workers)
* [Accepting and deleting HITs... careful with these](#Accepting-and-deleting-HITs...-careful-with-these)


In [1]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2


import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pylab as plt
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc dark solarized
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['figure.figsize'] = (16.0, 10.0)


import re
import pickle
import boto
from copy import deepcopy
import json
import os
import jinja2
import random
from tqdm import tqdm

from IPython.core.display import HTML

from boto.mturk.qualification import PercentAssignmentsApprovedRequirement, Qualifications, Requirement

from keysTkingdom import mturk_ai2
from keysTkingdom import aws_tokes
from keysTkingdom import mturk_aristo


from amt_utils.mturk import MTurk
from amt_utils.mturk import pickle_this, unpickle_this

# Setup

## estimate cost

In [2]:
cost_per_hit = 0.04
n_turkers_per_hit = 3
n_hits_per_video = 3

n_videos_low = 200
n_videos_high = 50000

low = cost_per_hit * n_turkers_per_hit * n_videos_low * n_hits_per_video
high = cost_per_hit * n_turkers_per_hit * n_videos_high * n_hits_per_video
print(str(low) + ' - $' + str(high))

72.0 - $18000.0


## load dataset

In [55]:
s3_base_path = 'https://s3-us-west-2.amazonaws.com/ai2-vision-animation-gan/annotation_data/subtask_frames/'


data_file_path = '../../build_dataset/ds_interface_bonus_skeleton.json'

with open(data_file_path, 'r') as f:
    ds_skeleton = json.load(f)

stills_to_annotate = []
for clip in ds_skeleton:
    for still in clip['keyFrames']:
        stills_to_annotate.append(still)
len(stills_to_annotate)

9000

In [464]:
subtask_stills = unpickle_this('subtask_stills_to_anno.pkl')

In [465]:
len(subtask_stills)

190

# Submitting HITs

In [43]:
from amt_utils.flintstones import prepare_hit
from amt_utils.flintstones import prepare_simpler_hit
from amt_utils.flintstones import write_task_page
from amt_utils.flintstones import generate_simpler_task_page

from amt_utils.mturk import expected_cost

In [539]:
turk_account = mturk_ai2
rw_host='mechanicalturk.amazonaws.com'
amt_con = MTurk(turk_account.access_key, turk_account.access_secret_key, host=rw_host)
# amt_con = MTurk(turk_account.access_key, turk_account.access_secret_key)
amt_con.get_account_balance()

$4,000.36

In [142]:
static_params = {
    'title': "Annotate characters from an animation frame - bonus potential",
    'description': "Draw bounding boxes and label characters appearing in a image. There is a bonus for writing labels for certain characters.",
    'keywords': ['animation', 'image', 'bounding box','bounding boxes', 'image annotation', 'bonus'],
    'frame_height': 1000,
    'amount': 0.01,
    'duration': 3600 * 1,
    'lifetime': 3600 * 24 * 2,
    'max_assignments': 3,
}

In [540]:
subtask_params = {
    'title': 'Annotate characters from an animation frame',
    'description': 'Draw a bounding box for a character in a image.',
    'keywords': ['animation', 'image', 'bounding box','bounding boxes', 'image annotation'],
    'frame_height': 1000,
    'amount': 0.01,
    'duration': 3600 * 1,
    'lifetime': 3600 * 24 * 2,
    'max_assignments': 3,
}

In [541]:
build_hit_group = []
for img_id, n_chars in tqdm(subtask_stills.items()):
    build_hit_group.extend(prepare_simpler_hit(s3_base_path, img_id, n_chars, subtask_params))

100%|██████████| 190/190 [00:00<00:00, 227.56it/s]


In [542]:
len(build_hit_group)

267

In [543]:
# build_hit_group = [prepare_hit(s3_base_path, img_id, subtask_params) for img_id in tqdm(stills_to_annotate[:5])]
# build_hit_group[0]['html'] = build_hit_group[0]['html'].replace( r'\u201c', 'test')
write_task_page(build_hit_group[0]['html'])

In [544]:
expected_cost(build_hit_group, static_params, amt_con)

8.01

**Procedure**

1. assure interface template is working

2. build and load ds skeleton for the sample to be submitted

3. make sure to sync images to s3

4. double check static params and host

5. build and submit

In [545]:
hit_group = [amt_con.create_html_hit(single_hit) for single_hit in tqdm(build_hit_group)]

100%|██████████| 267/267 [01:13<00:00,  4.44it/s]


last frame 6456

# Retrieve results

In [546]:
# from amt_utils.flintstones import get_assignments
from amt_utils.flintstones import filter_hits_by_date
from amt_utils.flintstones import filter_hits_by_completion
from amt_utils.flintstones import filter_hits_by_status
from amt_utils.flintstones import get_completed_hits
from amt_utils.mturk import pickle_this

In [548]:
all_hits = amt_con.get_all_hits()

In [551]:
# recent_hits = filter_hits_by_date(all_hits, 6)
recent_hits = filter_hits_by_date(all_hits, 9)
recent_hits = filter_hits_by_status(recent_hits)

In [552]:
len(recent_hits)

266

In [558]:
results = get_assignments(amt_con.connection, recent_hits)

In [538]:
results['3MJ28H2Y1F8IFBIMC6SVUPCCJLO5OI'][0].answers[0][0].fields

[u'{"stillID":"s_02_e_09_shot_013583_013657_taskb.png","characterBoxes":"[{\\"left\\":148,\\"top\\":134,\\"width\\":255,\\"height\\":346,\\"label\\":\\"s_02_e_09_shot_013583_013657_char_0_taskb.png\\"},{\\"left\\":784,\\"top\\":132,\\"width\\":258,\\"height\\":348,\\"label\\":\\"s_02_e_09_shot_013583_013657_char_0_taskb.png\\"}]"}']

In [537]:
results

defaultdict(list,
            {u'31D0ZWOD0BZUI02K5HH1CZWGDW9A03': [],
             u'341H3G5YF1EVJ8ASH9X0TSNPMHG0ZU': [],
             '3AA88CN98Q3XR87RPG5ZY5TWULDYK2': [],
             u'3MGHRFQY2MPVEI0NLNVHFLENGNC0Y3': [],
             u'3MJ28H2Y1F8IFBIMC6SVUPCCJLO5OI': [<boto.mturk.connection.Assignment at 0x11d8ec610>],
             u'3TC2K6WK9H2NVMCJN611ZWJ0K6T82H': [<boto.mturk.connection.Assignment at 0x11b545590>]})

In [559]:
pickle_this(results, 'turker_batch_6_subtask.pkl')

# Interact with workers

## reject assignments and ban workers

In [32]:
def ban_bad_workers(mturk_connection, worker_ids):
    for worker in worker_ids:
        reason_for_block = """
        Your HITs contained many incorrect character descriptions.
        """
        print('blocking ' + str(worker))
        mturk_connection.block_worker(worker, reason_for_block)

In [13]:
workers_to_ban = 'A300BK2S2PO3Q7'

In [33]:
ban_bad_workers(amt_con.connection, [workers_to_ban])

blocking A300BK2S2PO3Q7


In [14]:
to_reject = unpickle_this('to_reject.pkl')

In [15]:
def reject_assignments(mturk_connection, assignments_to_reject):
    feedback_message = """
    Your HITs contained many incorrect character descriptions.
    """
    reject_count = len(assignments_to_reject)
    for assignment_id in assignments_to_reject:
        try:
            mturk_connection.reject_assignment(assignment_id, feedback_message)
        except boto.mturk.connection.MTurkRequestError:
            print('assignment ' + str(assignment_id) + ' already accepted or rejected')

    return reject_count

In [31]:
reject_assignments(amt_con.connection, to_reject)

assignment 37Q970SNZF81C57Q82CTMS6JCZ11S0 already accepted or rejected
assignment 3ATTHHXXWBOK8SPNAJ3MDZ4X8GDXIV already accepted or rejected


150

## pay bonuses

In [16]:
subject = "More  HITs are available"
message = """
Hello, 

"""

In [15]:
# _ = mturk.notify_workers(good_workers[:20], subject, message)

In [46]:
bonuses = unpickle_this('bonuses_to_pay.pkl')

In [38]:
sum(bonuses.values()) * 0.01

18.94

In [53]:
def pay_bonuses(bonuses_to_pay):
    total_payed = 0
    bonus_reason_template = 'For Flinstones character labels written.'
    for aid, vals in tqdm(bonuses_to_pay.items()):
        n_chars = vals['count']
        wid = vals['worker_id']
        bonus_ammount = boto.mturk.price.Price(0.01 * n_chars)
        total_payed += bonus_ammount.amount
        bonus_reason = bonus_reason_template
        amt_con.connection.grant_bonus(wid, aid, bonus_ammount, bonus_reason)
    return total_payed

In [54]:
pay_bonuses(bonuses)

100%|██████████| 1306/1306 [05:30<00:00,  4.12it/s]


18.939999999999987

# Accepting and deleting HITs

Uncomment only when ready to accept or delete hits

reject assignments carefully

In [19]:
assignments =[]
for assignment_triple in list(results.values()):
    assignments.extend(assignment_triple)

len(assignments)

6222

In [20]:
assignments_to_accept = [asgmt for asgmt in assignments if asgmt.AssignmentId not in to_reject]

In [21]:
len(assignments_to_accept)

6072

In [27]:
e_count = 0
for assignment in tqdm(assignments_to_accept):
    try:
        amt_con.approve_assignment(assignment)
    except boto.mturk.connection.MTurkRequestError as e:
        e_count += 1

100%|██████████| 6072/6072 [24:11<00:00,  4.72it/s]   


In [ ]:
# _ = [amt_con.disable_hit(hit) for hit in hits_today]

In [ ]:
# number_rejected_assignments, number_rejected_workers = amt_util.reject_assignments(mturk, workers_to_ban, combined_consensus_with_workerid_df)
# print 'rejecting ' + str(number_rejected_assignments) + ' assignments' + ' from ' + str(number_rejected_workers) + ' workers'

In [147]:
amt_con.get_account_balance()

$10,000.00

In [525]:
amt_con.delete_all_hits()